In [ ]:
import pandas as pd

In [ ]:
zelenogorsk = pd.read_csv('../data/gkh_clear_Zelenogorsk.csv', encoding='utf-8', sep=',')
kansk = pd.read_csv('../data/gkh_clear_Kansk.csv', encoding='utf-8', sep=',')
achinsk = pd.read_csv('../data/gkh_clear_Achinsk.csv', encoding='utf-8', sep=',')

In [ ]:
# Используем две библиотеки для геокодирования:
# 1 - геокодеры из проекта geopy
from geopy.geocoders import Nominatim, Yandex
# 2 - стандартная библиотека веб-запросов (для работы с geocode.xyz)
import requests

# И еще парочку:
# - для ограничения скорости запросов
from geopy.extra.rate_limiter import RateLimiter
# - для визуализации процесса
from tqdm import tqdm

In [ ]:
import yaml

with open("../api_keys/geocoding_api.yml", "r") as f:
    api_keys = yaml.safe_load(f)

In [ ]:
geolocator_osm = Nominatim(user_agent="py")
geolocator_ya = Yandex(user_agent = "py", api_key = api_keys.get("yandex_geocoding_api").get("api_key"))
geocode_osm = RateLimiter(geolocator_osm.geocode, min_delay_seconds=2)
geocode_ya = RateLimiter(geolocator_ya.geocode, min_delay_seconds=2)

In [ ]:
zelenogorsk['addr_osm'] = ''
zelenogorsk['lon_osm'] = ''
zelenogorsk['lat_osm'] = ''
zelenogorsk['addr_ya'] = ''
zelenogorsk['lon_ya'] = ''
zelenogorsk['lat_ya'] = ''

In [ ]:
kansk['addr_osm'] = ''
kansk['lon_osm'] = ''
kansk['lat_osm'] = ''
kansk['addr_ya'] = ''
kansk['lon_ya'] = ''
kansk['lat_ya'] = ''

In [ ]:
achinsk['addr_osm'] = ''
achinsk['lon_osm'] = ''
achinsk['lat_osm'] = ''
achinsk['addr_ya'] = ''
achinsk['lon_ya'] = ''
achinsk['lat_ya'] = ''

In [ ]:
zelenogorsk_geo = pd.read_csv('../data/geocoded_zelenogorsk.csv', encoding='utf-8', sep=',')
kansk_geo = pd.read_csv('../data/geocoded_kansk.csv', encoding='utf-8', sep=',')
achinsk_geo = pd.read_csv('../data/geocoded_achinsk.csv', encoding='utf-8', sep=',')
krasnoyarsk_geo = pd.read_csv('../data/geocoded_krasnoyarsk.csv', encoding='utf-8', sep=',')

In [ ]:
krasnoyarsk_geo = pd.read_csv('../data/gkh_clear_Krasnoyarsk.csv', encoding='utf-8', sep=',')
krasnoyarsk_geo['addr_osm'] = ''
krasnoyarsk_geo['lon_osm'] = ''
krasnoyarsk_geo['lat_osm'] = ''
krasnoyarsk_geo['addr_ya'] = ''
krasnoyarsk_geo['lon_ya'] = ''
krasnoyarsk_geo['lat_ya'] = ''
krasnoyarsk_geo.to_csv('../data/geocoded_krasnoyarsk.csv')

In [ ]:
import json
from datetime import date

In [ ]:
with open("../state/state.json", "r") as infile:
    geocoding_state = json.load(infile)
geocoding_state

In [ ]:
task = pd.read_csv('../data/geocoded_' + geocoding_state['name'] + '.csv', encoding='utf-8', sep=',')

In [ ]:
if(date.fromisoformat(geocoding_state['last_date']) < date.today()):
    print('ok')
else:
    print('завтра')

In [ ]:
with open("../state/state.json", "r") as infile:
    geocoding_state = json.load(infile)

if(date.fromisoformat(geocoding_state['last_date']) < date.today()):
    print('Продолжаем парсить ', geocoding_state['name'], 'с индекса ', geocoding_state['parsed'])
    # task = krasnoyarsk_geo
    startIndex = geocoding_state['parsed'] #1000-512-2 # 2022-12-04
    endIndex = startIndex + 998 # 2022-12-04
    for index, row in task.iterrows():
        if (index >= startIndex) and (index < endIndex):
            name_for_osm = str.replace(row['formalname_city'] + ', ' + str(row['formalname_street']) + ', ' + str(row['house_number']) + str(row['building']) + str(row['block']) + str(row['letter']), 'nan', '')
            location_osm = geocode_osm(name_for_osm)
            print(name_for_osm)
            print('osm: ' + str(location_osm))
            location_ya = geocode_ya(row['address'])
            print(row['address'])
            print('ya: ' + str(location_ya))
            if location_osm:
                task.loc[index, 'addr_osm'] = location_osm.address
                task.loc[index, 'lon_osm'] = location_osm.longitude
                task.loc[index, 'lat_osm'] = location_osm.latitude
                # print(location_osm.address)
                # print(str(location_osm.longitude) + ' ' + str(location_osm.latitude))
            if location_ya:
                task.loc[index, 'addr_ya'] = location_ya.address
                task.loc[index, 'lon_ya'] = location_ya.longitude
                task.loc[index, 'lat_ya'] = location_ya.latitude
                # print(location_ya.address)
                # print(str(location_ya.longitude) + ' ' + str(location_ya.latitude))
        # else:
        #     break;

    task.to_csv('../data/geocoded_krasnoyarsk.csv')

    geocoding_state['parsed'] = endIndex
    geocoding_state['last_date'] = date.today().isoformat()
    with open("../state/state.json", "w") as outfile:
        json.dump(geocoding_state, outfile, indent=2)
else :
    print('Подожди до завтра')

In [ ]:
geocoding_state = {
    'name': 'krasnoyarsk',
    'parsed' : endIndex,
    'last_date' : date.today().isoformat()
}
with open("../state/state.json", "w") as outfile:
    json.dump(geocoding_state, outfile, indent=2)

## Serialization

In [ ]:
import json
from datetime import date

In [ ]:
date(2022, 12, 4).isoformat()

In [ ]:
date.fromisoformat('2022-12-04')

In [ ]:
date.today()

In [ ]:
geocoding_state = {
    'name': 'Krasnoyarsk',
    'parsed' : 1486,
    'last_date' : date.today().isoformat()
}
with open("../state/state.st", "w") as outfile:
    json.dump(geocoding_state, outfile, indent=2)

In [ ]:
with open("../state/state.st", "r") as infile:
    temp= json.load(infile)

In [ ]:
temp

In [ ]:
temp['name']

In [ ]:
from tqdm import tqdm

j = 0
for i in tqdm(range(20000000)):
    j = j + 1

print(i)